<a href="https://colab.research.google.com/github/Werricsson-Santos/transfer-learning/blob/master/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer learning / fine-tuning

Este tutorial irá guiá-lo pelo processo de usar aprendizado por transferência (transfer learning) para criar um classificador de imagens preciso a partir de um número relativamente pequeno de amostras de treinamento. De maneira geral, aprendizado por transferência refere-se ao processo de aproveitar o conhecimento aprendido em um modelo para treinar outro modelo.

Mais especificamente, o processo envolve utilizar uma rede neural existente, previamente treinada com bom desempenho em um conjunto de dados maior, como base para um novo modelo que aproveite a precisão da rede anterior para uma nova tarefa. Este método tornou-se popular nos últimos anos para melhorar o desempenho de redes neurais treinadas em conjuntos de dados pequenos. A ideia intuitiva é que o novo conjunto de dados pode ser pequeno demais para alcançar um bom desempenho por si só, mas sabemos que a maioria das redes neurais treinadas para aprender características de imagens frequentemente aprende características similares, especialmente nas camadas iniciais, onde são mais genéricas (detectores de bordas, padrões básicos, etc.).

O aprendizado por transferência foi amplamente viabilizado pela disponibilização pública de modelos de última geração; para os modelos de melhor desempenho em tarefas de classificação de imagens (como no ILSVRC), é prática comum não apenas publicar a arquitetura, mas também liberar os pesos treinados do modelo. Isso permite que amadores utilizem esses classificadores de imagens avançados para aumentar o desempenho de seus próprios modelos específicos.

Extração de características vs. ajuste fino
Em um extremo, o aprendizado por transferência pode envolver o uso da rede pré-treinada com seus pesos congelados, utilizando uma de suas camadas ocultas (geralmente a última) como extrator de características e empregando essas características como entrada para uma rede neural menor.

No outro extremo, começamos com a rede pré-treinada, mas permitimos que alguns pesos (geralmente os da última camada ou últimas camadas) sejam modificados. Esse procedimento também é conhecido como "ajuste fino" (fine-tuning), porque ajustamos levemente os pesos da rede pré-treinada para a nova tarefa. Geralmente treinamos essa rede com uma taxa de aprendizado mais baixa, pois presumimos que as características já estão relativamente boas e não precisam de grandes mudanças.

Às vezes, fazemos algo intermediário: congelamos apenas as camadas iniciais/genéricas, mas ajustamos as camadas finais. Qual estratégia é melhor depende do tamanho do seu conjunto de dados, do número de classes e do quanto ele se assemelha ao conjunto de dados no qual o modelo anterior foi treinado (e, assim, se pode se beneficiar dos mesmos extratores de características aprendidos). Uma discussão mais detalhada sobre como escolher a melhor estratégia pode ser encontrada em [1] [2].

Procedimento
Neste guia, abordaremos o processo de carregar um classificador de imagens de última geração, o VGG16, que venceu o desafio ImageNet em 2014, e utilizá-lo como um extrator de características fixo para treinar um classificador personalizado menor com nossas próprias imagens. Com apenas algumas alterações no código, também é possível experimentar o ajuste fino.

Primeiramente, carregaremos o VGG16 e removeremos sua camada final, a camada de classificação softmax de 1000 classes específica do ImageNet, substituindo-a por uma nova camada de classificação para as classes que estamos treinando. Em seguida, congelaremos todos os pesos da rede, exceto os novos conectados à camada de classificação, e treinaremos essa camada de classificação sobre nosso novo conjunto de dados.

Também compararemos este método com o treinamento de uma pequena rede neural do zero no novo conjunto de dados. Como veremos, isso melhorará drasticamente nossa precisão. Começaremos com essa parte.

Como exemplo, usaremos um conjunto de dados com cerca de 6000 imagens pertencentes a 97 classes, e treinaremos um classificador de imagens com aproximadamente 80% de precisão. Vale notar que essa estratégia escala bem para conjuntos de imagens nos quais você pode ter apenas algumas centenas ou menos de imagens. O desempenho será menor com poucas amostras (dependendo das classes), como esperado, mas ainda impressionante considerando as restrições habituais.

In [ ]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

## Obtendo um conjunto de dados

O primeiro passo será carregar nossos dados. Como exemplo, utilizaremos o conjunto de dados cats_vs_dogs, que contém imagens rotuladas pertencentes a essas duas categorias de animais.

Para obter este conjunto de dados, você pode executar o script de download chamado download.sh na pasta de dados ou usar os seguintes comandos:

In [ ]:
!echo "Downloading cats_vs_dogs for image notebooks"
!curl -L -o cats_vs_dogs.zip --progress-bar https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
!unzip cats_vs_dogs.zip
!rm cats_vs_dogs.zip
!ls

In [ ]:
root = 'PetImages'

train_split, val_split = 0.7, 0.15

categories = [os.path.join(root, folder) for folder in os.listdir(root) if os.path.isdir(os.path.join(root, folder))]

print(categories)



Esta função é útil para pré-processar os dados em uma imagem e um vetor de entrada.

In [ ]:
def get_image(path):
    try:
        img = image.load_img(path, target_size=(224, 224))  # Redimensiona a imagem
        x = image.img_to_array(img)  # Converte para array
        x = np.expand_dims(x, axis=0)  # Adiciona dimensão extra
        x = preprocess_input(x)  # Pré-processa a imagem
        return img, x
    except Exception as e:
        print(f"Erro ao processar a imagem {path}: {e}")
        return None, None


Carregar todas as imagens da pasta raiz.

In [ ]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]

    images = images[:1000]

for img_path in images:
    img, x = get_image(img_path)
    if img is not None:
        data.append({'x': np.array(x), 'y': c})

# count the number of classes
num_classes = len(categories)

Aleatorizar a ordem dos dados.

In [ ]:
random.shuffle(data)

Criar divisão de treinamento/validação/teste (70%, 15%, 15%).

In [ ]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

Separar os dados dos rótulos.

In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

Pré-processar os dados como antes, garantindo que sejam do tipo float32 e normalizados entre 0 e 1.

In [ ]:
# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

Vamos obter um resumo do que temos.

In [ ]:
# summary
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)

If everything worked properly, you should have loaded a bunch of images, and split them into three sets: train, val, and test. The shape of the training data should be (n, 224, 224, 3) where n is the size of your training set, and the labels should be (n, c) where c is the number of classes (97 in the case of 101_ObjectCategories.

Notice that we divided all the data into three subsets -- a training set train, a validation set val, and a test set test. The reason for this is to properly evaluate the accuracy of our classifier. During training, the optimizer uses the validation set to evaluate its internal performance, in order to determine the gradient without overfitting to the training set. The test set is always held out from the training algorithm, and is only used at the end to evaluate the final accuracy of our model.

Let's quickly look at a few sample images from our dataset.

In [ ]:
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(16,4))
plt.imshow(concat_image)

In [ ]:
# Remove a dimensão extra
x_train = np.squeeze(x_train, axis=1)
x_val = np.squeeze(x_val, axis=1)
x_test = np.squeeze(x_test, axis=1)

## Primeiro, treinando uma rede neural do zero
Antes de realizar o aprendizado por transferência, vamos primeiro construir uma rede neural do zero para classificar nosso conjunto de dados. Isso nos dará uma linha de base para compararmos com a rede ajustada por transferência posteriormente.

A rede que construiremos conterá 4 camadas alternadas de convolução e max-pooling, seguidas por um dropout após cada par de camadas de convolução/pooling. Após a última camada de pooling, anexaremos uma camada totalmente conectada (fully-connected) com 256 neurônios, outra camada de dropout e, por fim, uma camada de classificação softmax para nossas classes.

Nossa função de perda será, como de costume, a perda de entropia cruzada categórica (categorical cross-entropy loss), e nosso algoritmo de aprendizado será o AdaDelta. Diversos aspectos dessa rede podem ser alterados para melhorar o desempenho — talvez usar uma rede maior ou um otimizador diferente possa ajudar — mas, para os propósitos deste notebook, o objetivo é apenas entender uma linha de base aproximada para fins de comparação. Portanto, não é necessário gastar muito tempo otimizando esta rede.

Após compilar a rede, executaremos model.summary() para obter um instantâneo de suas camadas.

In [ ]:
# build the network
model = Sequential()
print("Input dimensions: ",x_train.shape[1:])

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Criamos uma rede de tamanho médio com aproximadamente 1,2 milhões de pesos e vieses (os parâmetros). A maioria desses parâmetros está conectada à camada totalmente conectada antes da ativação softmax, chamada de "dense_5".

Agora podemos prosseguir e treinar nosso modelo por 100 épocas, com um tamanho de lote (batch size) de 128. Também registraremos o histórico do treinamento para que possamos plotar a perda ao longo do tempo posteriormente.

In [ ]:
# compile the model to use categorical cross-entropy loss function and adadelta optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))


Vamos plotar a perda de validação (validation loss) e a precisão de validação (validation accuracy) ao longo do tempo.

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_accuracy"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

## Aprendizado por transferência usando uma rede existente
Agora podemos passar para a estratégia principal de treinar um classificador de imagens em nosso pequeno conjunto de dados: começar com uma rede maior e já treinada.

Para começar, carregaremos a rede VGG16 do Keras, que foi treinada no conjunto de dados ImageNet, com os pesos salvos online. Se esta for a primeira vez que você carrega o VGG16, será necessário aguardar um pouco para que os pesos sejam baixados da web.

Depois que a rede for carregada, podemos inspecionar novamente as camadas usando o método summary().

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

Observe que o VGG16 é muito maior que a rede que construímos anteriormente. Ele contém 13 camadas convolucionais e duas camadas totalmente conectadas no final, totalizando mais de 138 milhões de parâmetros, cerca de 100 vezes mais parâmetros do que a rede que fizemos antes. Assim como na nossa primeira rede, a maior parte dos parâmetros está nas conexões que levam à primeira camada totalmente conectada.

O VGG16 foi projetado para resolver o conjunto de dados ImageNet e atinge uma taxa de erro top-5 de 8,8%. Isso significa que 91,2% das amostras de teste foram classificadas corretamente dentro das 5 principais previsões para cada imagem. Sua precisão top-1 — equivalente à métrica de precisão que temos usado (quando a principal previsão é correta) — é de 73%. Este desempenho é especialmente impressionante, considerando que existem 1000 classes, não apenas 97, e que palpites aleatórios resultariam em apenas 0,1% de precisão.

Para usar essa rede em nossa tarefa, "removemos" a camada final de classificação, a camada softmax de 1000 neurônios correspondente ao ImageNet, e a substituímos por uma nova camada softmax para nosso conjunto de dados, que contém 97 neurônios no caso do conjunto 101_ObjectCategories.

Implementação:
É mais prático criar uma cópia do VGG16 desde sua camada de entrada até a penúltima camada e trabalhar com isso, em vez de modificar o objeto VGG diretamente. Tecnicamente, nunca "removemos" nada, apenas contornamos ou ignoramos. Isso pode ser feito com a classe Model do Keras, para inicializar um novo modelo cuja camada de entrada é a mesma do VGG16, mas cuja camada de saída é nossa nova camada softmax, chamada new_classification_layer.

Nota:
Embora pareça que estamos duplicando essa grande rede, internamente o Keras apenas copia todas as camadas por referência. Portanto, não precisamos nos preocupar com sobrecarga de memória.

In [ ]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)

We are going to retrain this network, model_new on the new dataset and labels. But first, we need to freeze the weights and biases in all the layers in the network, except our new one at the end, with the expectation that the features that were learned in VGG should still be fairly relevant to the new image classification task. Not optimal, but most likely better than what we can train to in our limited dataset.

By setting the trainable flag in each layer false (except our new classification layer), we ensure all the weights and biases in those layers remain fixed, and we simply train the weights in the one layer at the end. In some cases, it is desirable to not freeze all the pre-classification layers. If your dataset has enough samples, and doesn't resemble ImageNet very much, it might be advantageous to fine-tune some of the VGG layers along with the new classifier, or possibly even all of them. To do this, you can change the below code to make more of the layers trainable.

In the case of CalTech-101, we will just do feature extraction, fearing that fine-tuning too much with this dataset may overfit. But maybe we are wrong? A good exercise would be to try out both, and compare the results.

So we go ahead and freeze the layers, and compile the new model with exactly the same optimizer and loss function as in our first network, for the sake of a fair comparison. We then run summary again to look at the network's architecture.

In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

Ao observar o sumário, vemos que a rede é idêntica ao modelo VGG que instanciamos anteriormente, exceto pela última camada. Antes, ela era uma softmax com 1000 neurônios, e agora foi substituída por uma nova softmax com 97 neurônios. Além disso, ainda temos cerca de 134 milhões de pesos, mas a grande maioria deles agora são "parâmetros não treináveis", porque congelamos as camadas onde eles estão contidos.

Agora, temos apenas 397.000 parâmetros treináveis, o que é, na verdade, apenas um quarto do número de parâmetros necessários para treinar o primeiro modelo.

Treinamento:
Como antes, seguimos em frente e treinamos o novo modelo usando os mesmos hiperparâmetros (tamanho do lote e número de épocas) de antes, junto com o mesmo algoritmo de otimização. Também registramos o histórico durante o treinamento.

In [ ]:
history2 = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))

Neste caso do treinamento com as imagens de gatos e cachorros extraídas do kaggle, podemos notar que foi mais vantajoso treinar a rede do zero. Por mais que o aprendizado por transferência tenha o mesmo resultado, o processo de treinamento acaba se tornando mais lento.

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.plot(history2.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_accuracy"])
ax2.plot(history2.history["val_accuracy"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
img, x = get_image('PetImages/Dog/10002.jpg')
probabilities = model_new.predict([x])